In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [3]:
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

config = SparkConf().setMaster('local').setAppName('lab53')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

In [4]:
orders_list = [("01", "02", 350, 1),
                ("01", "04", 580, 1),
                ("01", "07", 320, 2),
                ("02", "03", 450, 1),
                ("02", "06", 220, 1),
                ("03", "01", 195, 1),
                ("04", "09", 270, 3),
                ("04", "08", 410, 2),
                ("05", "02", 350, 1)]

# Tạo Dataframe
order_df = spark.createDataFrame(orders_list).toDF("order_id","pro_id","unit_price","qty")
order_df.show()

product_list = [("01", "Scroll Mouse", 250, 20),
                ("02", "Optical Mouse", 350, 20),
                ("03", "Wireless Mouse", 450, 50),
                ("04", "Wireless Keyboard", 580, 50),
                ("05", "Standard Keyboard", 360, 10),
                ("06", "16 GB Flash Storage", 240, 100),
                ("07", "32 GB Flash Storage", 320, 50),
                ("08", "64 GB Flash Storage", 430, 25)]

# Tạo Dataframe
product_df = spark.createDataFrame(product_list).toDF("pro_id","pro_name","list_price","qty")
product_df.show()

+--------+------+----------+---+
|order_id|pro_id|unit_price|qty|
+--------+------+----------+---+
|      01|    02|       350|  1|
|      01|    04|       580|  1|
|      01|    07|       320|  2|
|      02|    03|       450|  1|
|      02|    06|       220|  1|
|      03|    01|       195|  1|
|      04|    09|       270|  3|
|      04|    08|       410|  2|
|      05|    02|       350|  1|
+--------+------+----------+---+

+------+-------------------+----------+---+
|pro_id|           pro_name|list_price|qty|
+------+-------------------+----------+---+
|    01|       Scroll Mouse|       250| 20|
|    02|      Optical Mouse|       350| 20|
|    03|     Wireless Mouse|       450| 50|
|    04|  Wireless Keyboard|       580| 50|
|    05|  Standard Keyboard|       360| 10|
|    06|16 GB Flash Storage|       240|100|
|    07|32 GB Flash Storage|       320| 50|
|    08|64 GB Flash Storage|       430| 25|
+------+-------------------+----------+---+



In [10]:
# Xác định điều kiện Join
join_expr = order_df.pro_id == product_df.pro_id

product_renamed_df = product_df.withColumnRenamed("qty", "reorder_qty")

# Join hai Dataframe
order_df.join(product_renamed_df, join_expr, 'inner') \
    .drop(product_renamed_df.pro_id) \
    .select("order_id", "pro_id", "pro_name", "unit_price", "list_price", "qty") \
    .show()

+--------+------+-------------------+----------+----------+---+
|order_id|pro_id|           pro_name|unit_price|list_price|qty|
+--------+------+-------------------+----------+----------+---+
|      03|    01|       Scroll Mouse|       195|       250|  1|
|      01|    02|      Optical Mouse|       350|       350|  1|
|      05|    02|      Optical Mouse|       350|       350|  1|
|      02|    03|     Wireless Mouse|       450|       450|  1|
|      01|    04|  Wireless Keyboard|       580|       580|  1|
|      02|    06|16 GB Flash Storage|       220|       240|  1|
|      01|    07|32 GB Flash Storage|       320|       320|  2|
|      04|    08|64 GB Flash Storage|       410|       430|  2|
+--------+------+-------------------+----------+----------+---+

